In [1]:
import sys
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
from torchvision import models
from torchvision.models.resnet import BasicBlock, _resnet
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
import math
from collections import Counter
import statistics
from PIL import Image
import krippendorff
import random

import statsmodels.api as sm
import statsmodels.stats.api as smi
import torch.nn.functional as F
from torchsummary import summary
from sklearn.metrics import mean_absolute_error


In [2]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.is_available())
print(torch.__version__)
print(torch.version.cuda)

cuda:1
True
2.0.1
11.8


In [3]:
#train val split

In [4]:
#list of all files
import os
annos_directory_path = '/media/scratch/datasets/voc2007/VOCdevkit/VOC2007/annos/'
annos_file_list = os.listdir(annos_directory_path)

In [5]:
image_directory_path = "/media/scratch/datasets/voc2007/VOCdevkit/VOC2007/JPEGImages"
image_file_list = os.listdir(image_directory_path)

In [6]:
#names of all training files
train_file = '/media/scratch/datasets/voc2007/VOCdevkit/VOC2007/ImageSets/Main/train.txt'
with open(train_file, 'r') as file:
    train_file_names = [line.strip() for line in file]

In [7]:
#names of all val files
val_file = '/media/scratch/datasets/voc2007/VOCdevkit/VOC2007/ImageSets/Main/val.txt'
with open(val_file, 'r') as file:
    val_file_names = [line.strip() for line in file]

In [8]:
#split image files into train and val
train_image_files = []
val_image_files = []
for index,file in enumerate(image_file_list):
    file_name = file.replace(".jpg","")
    if file_name in train_file_names:
        train_image_files.append(file)
    elif file_name in val_file_names:
        val_image_files.append(file)

In [9]:
#split label files into train and val
train_label_files = []
val_label_files = []
for index,file in enumerate(annos_file_list):
    file_name = file.replace(".txt","")
    if file_name in train_file_names:
        train_label_files.append(file)
    elif file_name in val_file_names:
        val_label_files.append(file)

In [10]:
#creating labels and probability distributions

In [11]:
#reads annos for train
train_annos = []
for file_anno in train_label_files:
    file_path = os.path.join(annos_directory_path, file_anno)
    with open(file_path, mode = 'r', encoding = 'utf-8-sig') as file:
        lines = file.readlines()

    single_image_annos = []
    for line in lines:
        line = line.split(' ')
        line = [i.strip() for i in line]
        single_image_annos.append(line)
    
    train_annos.append(single_image_annos)

In [12]:
#gets only the classes for train labels
train_classes_annos = []
for j in range(len(train_annos)):
    only_class = []
    for i in train_annos[j]:
        only_class.append(i[0])
    train_classes_annos.append(only_class)

In [13]:
#reads annos for val
val_annos = []
for file_anno in val_label_files:
    file_path = os.path.join(annos_directory_path, file_anno)
    with open(file_path, mode = 'r', encoding = 'utf-8-sig') as file:
        lines = file.readlines()

    single_image_annos = []
    for line in lines:
        line = line.split(' ')
        line = [i.strip() for i in line]
        single_image_annos.append(line)
    
    val_annos.append(single_image_annos)

In [14]:
#gets only the classes for val labels
val_classes_annos = []
for j in range(len(val_annos)):
    only_class = []
    for i in val_annos[j]:
        only_class.append(i[0])
    val_classes_annos.append(only_class)

In [15]:
#class to number dictionary
class_dict = {
    'aeroplane': 0,
    'bicycle': 1,
    'bird': 2,
    'boat': 3,
    'bottle': 4,
    'bus': 5,
    'car': 6,
    'cat': 7,
    'chair': 8,
    'cow': 9,
    'diningtable': 10,
    'dog': 11,
    'horse': 12,
    'motorbike': 13,
    'person': 14,
    'pottedplant': 15,
    'sheep': 16,
    'sofa': 17,
    'train': 18,
    'tvmonitor': 19
}

In [16]:
#class to number conversion for train set
train_class_numbers = train_classes_annos
for annotation in range(len(train_classes_annos)):
    for j in range(len(train_classes_annos[annotation])):
        train_class_numbers[annotation][j] = class_dict.get(train_classes_annos[annotation][j])

In [17]:
#class to number conversion for val set
val_class_numbers = val_classes_annos
for annotation in range(len(val_classes_annos)):
    for j in range(len(val_classes_annos[annotation])):
        val_class_numbers[annotation][j] = class_dict.get(val_classes_annos[annotation][j])

In [18]:
def convert_to_encoded(sample):
    encoded = [0 for y in range(20)]
    for i in sample:
        encoded[i] = encoded[i] + 1
    return encoded

In [19]:
#encode train set
train_encoded_list = []
for i in range(len(train_class_numbers)):
    train_encoded_list.append(convert_to_encoded(train_class_numbers[i]))

In [20]:
print(train_encoded_list)

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0], [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0], [0, 0, 

In [21]:
#encode train set
val_encoded_list = []
for i in range(len(val_class_numbers)):
    val_encoded_list.append(convert_to_encoded(val_class_numbers[i]))

In [22]:
#convert train encoding to probabilities
train_probs = []
for i in train_encoded_list:
    total = sum(i)
    probs = [x / total for x in i]
    train_probs.append(probs)

In [23]:
#convert val encoding to probabilities
val_probs = []
for i in val_encoded_list:
    total = sum(i)
    probs = [x / total for x in i]
    val_probs.append(probs)

In [24]:
print(val_probs)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.75, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.75, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [25]:
import torch
import numpy as np
#train labels for annotators
num_classes = 20
num_annotators = 7
train_ann_labels = []
for i in range(len(train_probs)):
            new_label = np.random.choice(20,num_annotators, p=train_probs[i])
            train_ann_labels.append(new_label.tolist())

In [26]:
#val labels for annotators
val_ann_labels = []
for i in range(len(val_probs)):
            new_label = np.random.choice(20,num_annotators, p=val_probs[i])
            val_ann_labels.append(new_label.tolist())

In [27]:
train_labels = torch.tensor(train_ann_labels)
val_labels = torch.tensor(val_probs)

In [28]:
print(val_labels)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [29]:
print(val_labels.shape)

torch.Size([2510, 20])


In [30]:
'''import os
import numpy as np
from PIL import Image

def calculate_mean_std(dataset_dir):
    mean = np.zeros(3)
    std = np.zeros(3)
    num_samples = 0

    for root, dirs, files in os.walk(dataset_dir):
        for filename in files:
            if filename.endswith(".jpg"):
                img_path = os.path.join(root, filename)
                img = Image.open(img_path).convert("RGB")
                img = np.array(img) / 255.0  # Convert image to NumPy array and scale to [0, 1]

                mean += img.mean((0, 1))
                std += img.std((0, 1))
                num_samples += 1

    mean /= num_samples
    std /= num_samples

    return mean, std

custom_mean, custom_std = calculate_mean_std('/media/scratch/datasets/voc2007/VOCdevkit/VOC2007/JPEGImages')
print("Custom Mean:", custom_mean)
print("Custom Std:", custom_std)'''


'import os\nimport numpy as np\nfrom PIL import Image\n\ndef calculate_mean_std(dataset_dir):\n    mean = np.zeros(3)\n    std = np.zeros(3)\n    num_samples = 0\n\n    for root, dirs, files in os.walk(dataset_dir):\n        for filename in files:\n            if filename.endswith(".jpg"):\n                img_path = os.path.join(root, filename)\n                img = Image.open(img_path).convert("RGB")\n                img = np.array(img) / 255.0  # Convert image to NumPy array and scale to [0, 1]\n\n                mean += img.mean((0, 1))\n                std += img.std((0, 1))\n                num_samples += 1\n\n    mean /= num_samples\n    std /= num_samples\n\n    return mean, std\n\ncustom_mean, custom_std = calculate_mean_std(\'/media/scratch/datasets/voc2007/VOCdevkit/VOC2007/JPEGImages\')\nprint("Custom Mean:", custom_mean)\nprint("Custom Std:", custom_std)'

In [31]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4485348, 0.42499675, 0.39198778], std=[0.24029811, 0.23445981, 0.23704079])
])

transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4485348, 0.42499675, 0.39198778], std=[0.24029811, 0.23445981, 0.23704079])
])

In [32]:
import torch
file = train_image_files[0]
print(file)
file_path = '/media/scratch/datasets/voc2007/VOCdevkit/VOC2007/JPEGImages'
file_name = os.path.join(file_path,file)
img = Image.open(file_name)
#plt.imshow(img.permute(1,2,0))
print(img.size)

000012.jpg
(500, 333)


In [33]:
image_dir = '/media/scratch/datasets/voc2007/VOCdevkit/VOC2007/JPEGImages'
class VOC2007Dataset(Dataset):
        def __init__(self, image_dir,image_files, labels, transform):
            self.image_dir = image_dir
            self.image_files = image_files
            self.labels = labels
            self.transform = transform
        def __len__(self):
            return len(self.image_files)
        def __getitem__(self,index):
            image_name = self.image_files[index]
            image = Image.open(os.path.join(self.image_dir,image_name))
            #print(image.size)
            label = self.labels[index]

            if self.transform:
                image = self.transform(image)

            return image,label


In [34]:
train_set = VOC2007Dataset(image_dir,train_image_files,train_labels,transform = transform_train)
val_set = VOC2007Dataset(image_dir,val_image_files,val_labels,transform = transform_test)

In [35]:
print(val_set[2])

(tensor([[[-0.7079, -0.4957, -0.5773,  ..., -0.7895, -0.2183,  1.1689],
         [-0.6589, -0.3325,  0.1897,  ..., -0.3978,  0.4182, -0.3815],
         [-0.8711, -0.9037, -0.7242,  ...,  0.8914,  0.7446, -0.4957],
         ...,
         [-0.2020, -0.2020, -0.2020,  ..., -0.3978, -0.3652, -0.4141],
         [-0.1693, -0.0877, -0.2673,  ..., -0.3162, -0.2836, -0.3489],
         [-0.3815, -0.3815, -0.3162,  ..., -0.3162, -0.2346, -0.2673]],

        [[-0.2739, -0.2070, -0.2070,  ..., -0.2070,  0.1275,  1.5827],
         [-0.1568,  0.2112,  0.7631,  ..., -0.0732,  0.8468,  0.1777],
         [-0.2404, -0.3073, -0.1902,  ...,  1.0475,  0.9304, -0.2571],
         ...,
         [ 0.3617,  0.3450,  0.3784,  ...,  0.2781,  0.3784,  0.3450],
         [ 0.4621,  0.5122,  0.3283,  ...,  0.3617,  0.3952,  0.3617],
         [ 0.3115,  0.3115,  0.3115,  ...,  0.3617,  0.4621,  0.4286]],

        [[-0.7272, -0.6280, -0.6280,  ..., -0.7438,  0.1331,  1.7378],
         [-0.8430, -0.3963,  0.5301,  ..., -

In [36]:
print(train_set.labels)

tensor([[ 6,  6,  6,  ...,  6,  6,  6],
        [14, 12, 12,  ..., 14, 12, 12],
        [ 1,  1, 14,  ..., 14,  1, 14],
        ...,
        [ 8, 14, 17,  ..., 14, 14, 17],
        [ 6,  6,  6,  ...,  6,  6,  6],
        [11, 11, 11,  ..., 11, 11, 11]])


In [37]:
count = 0
for i in train_labels.tolist():
    for j in i:
        if j==19:
            count+=1
print(count)

506


In [38]:
multi_voc_TrainLoader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2) #DataLoader with Noise
multi_voc_ValLoader = DataLoader(val_set, batch_size=32, shuffle=True, num_workers=2) #Testing DataLoader

In [39]:
import torch
import torchvision.models as models
import torch.nn as nn

# Load the pretrained ResNet-50 model
baseline_model = models.resnet50(pretrained=True)

# Access the original fully connected layer
#original_fc = baseline_model.fc

# Specify the number of classes in your task
num_classes = 20

# Define a new fully connected layer
#new_fc = nn.Linear(in_features=2048, out_features=num_classes)

# Replace the original fully connected layer with the new one
#baseline_model.fc = new_fc
#baseline_model.to(device)

/home/kevin/anaconda3/envs/kevGPU/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kevin/anaconda3/envs/kevGPU/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
'''import PreResNet
baseline_model = PreResNet.ResNet18()
baseline_model.to(device)'''


in_ftrs = baseline_model.fc.in_features

class_outputs_layers = nn.ModuleList([nn.Linear(in_ftrs, num_classes)for i in range (num_annotators)])

#define model
class MultiTaskModel (nn.Module):
    def __init__(self, preloaded, class_outputs_layers,):
        super(MultiTaskModel, self).__init__()
        self.preloaded = nn.Sequential(*list(preloaded.children())[:-2])
        self.class_outputs_layers = class_outputs_layers

    def forward(self, x):
        out = self.preloaded(x)
        out = F.avg_pool2d(out, 7) 
        out = out.view(out.size(0), -1)
        class_outputs_logits = [layers(out) for layers in self.class_outputs_layers] #i think this code will calculate the logits for every single layer simultaneoulsy
        return class_outputs_logits


MultiTask_model = MultiTaskModel(baseline_model, class_outputs_layers).to(device)
print(in_ftrs)
MultiTask_model.to(device)

2048


MultiTaskModel(
  (preloaded): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Co

In [41]:
'''# Freeze all layers except the last linear layers
for name, param in MultiTask_model.named_parameters():
    if not name.startswith('class_outputs_layers'):
        param.requires_grad = False
    else:
        param.requires_grad = True
'''

"# Freeze all layers except the last linear layers\nfor name, param in MultiTask_model.named_parameters():\n    if not name.startswith('class_outputs_layers'):\n        param.requires_grad = False\n    else:\n        param.requires_grad = True\n"

In [42]:
#hyper parameters
num_epochs = 100
learning_rate = 0.001
pos_weight = torch.tensor([1,1,1,1,1,1,0.25,1,1,1,1,1,1,1,0.25,1,1,1,1,1]).to(device)
criterion = nn.CrossEntropyLoss(weight = pos_weight)
optimizer_full = torch.optim.SGD(MultiTask_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = lr_scheduler.StepLR(optimizer_full, step_size = 50, gamma=0.1)

In [43]:
train_set.labels[0]


tensor([6, 6, 6, 6, 6, 6, 6])

In [44]:
def customLossTrain(outputs, labels):
    loss = 0
    #print(labels[:,0])
    for index, batch in enumerate(outputs):
        #print(labels)
        ann_labels = labels[:,index]
        #print(ann_labels)
        #print(ann_labels.shape)
        loss_temp = criterion(batch, ann_labels)

        #optimizer_list[index].zero_grad()
        #loss_temp.backward(retain_graph=True)
        #optimizer_list[index].step()

        loss += loss_temp
    return loss/num_annotators

def customLossVal(outputs, labels):
    loss = 0
    for index, batch in enumerate(outputs):
        loss_temp = criterion(batch, labels)

        #optimizer_list[index].zero_grad()
        #loss_temp.backward(retain_graph=True)
        #optimizer_list[index].step()

        loss += loss_temp
    return loss/num_annotators

In [45]:
n_total_steps = len(multi_voc_TrainLoader)
epoch = 0
for epoch in range(num_epochs):
    loss_trainList = []
    loss_trainAvg = 0
    for step,(data, labels) in enumerate(multi_voc_TrainLoader):
        data = data.to(device) 
        labels = labels.to(device)
        #print(labels)


        #Forward pass
        optimizer_full.zero_grad()
        outputs = MultiTask_model(data) #outputs is a list for each task

        loss_sum = customLossTrain(outputs, labels)
        loss_trainList.append(loss_sum)

        loss_sum.backward()
        optimizer_full.step()

        if(step%10 == 9):
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{n_total_steps}], Loss: {loss_sum.item()}")
    scheduler.step()

    loss_trainAvg = sum(loss_trainList)/len(loss_trainList)

    print(f"Loss Train Avg: {loss_trainAvg}")
    print("completed training")
    print(" ")
    
    
    with torch.no_grad():
        mean_error_list = []
        total_hamm_loss = []
        for data, labels in multi_voc_ValLoader:  # Replace 'test_loader' with your DataLoader for the test set
            data = data.to(device)
            labels = labels.to(device)
            outputs = MultiTask_model(data)
            #print(sum(outputs).size)
            output_avg = sum(outputs)/len(outputs)
            
            predictions = torch.sigmoid(output_avg)
            mean_abs_error = mean_absolute_error(labels.flatten().cpu(), predictions.flatten().cpu())
            mean_error_list.append(mean_abs_error)
        mean_abs_error_complete = sum(mean_error_list)/len(mean_error_list)
    print(f"{epoch+1}: Mean Absolute Error: {mean_abs_error_complete}")
    print(" ")
     

Epoch [1/100], Step [10/79], Loss: 3.0054311752319336
Epoch [1/100], Step [20/79], Loss: 2.873810291290283
Epoch [1/100], Step [30/79], Loss: 2.8412959575653076
Epoch [1/100], Step [40/79], Loss: 2.771294116973877
Epoch [1/100], Step [50/79], Loss: 2.7031984329223633
Epoch [1/100], Step [60/79], Loss: 2.7844858169555664
Epoch [1/100], Step [70/79], Loss: 2.586747169494629
Loss Train Avg: 2.8027946949005127
completed training
 
1: Mean Absolute Error: 0.4707553428185137
 
Epoch [2/100], Step [10/79], Loss: 2.4078452587127686
Epoch [2/100], Step [20/79], Loss: 2.197869062423706
Epoch [2/100], Step [30/79], Loss: 2.5886120796203613
Epoch [2/100], Step [40/79], Loss: 2.0217676162719727
Epoch [2/100], Step [50/79], Loss: 2.170107364654541
Epoch [2/100], Step [60/79], Loss: 2.110846996307373
Epoch [2/100], Step [70/79], Loss: 1.9573370218276978
Loss Train Avg: 2.2181479930877686
completed training
 
2: Mean Absolute Error: 0.46045191642604294
 
Epoch [3/100], Step [10/79], Loss: 2.0840165615

In [ ]:
print(MultiTask_model(train_set[23][0].to(device).unsqueeze(0))[0].squeeze() + MultiTask_model(train_set[23][0].to(device).unsqueeze(0))[1].squeeze())
print(MultiTask_model(train_set[23][0].to(device).unsqueeze(0))[0].squeeze())
print(MultiTask_model(train_set[23][0].to(device).unsqueeze(0))[1].squeeze())

In [46]:
cool = MultiTask_model(train_set[23][0].to(device).unsqueeze(0))
print(torch.sigmoid(cool[0]))
print(cool[0][0])
for i in cool[0][0]:
    print(i)

tensor([[0.3274, 0.5149, 0.4704, 0.3046, 0.5017, 0.3605, 0.7633, 0.4479, 0.7060,
         0.2756, 0.3522, 0.5712, 0.5179, 0.5231, 0.9569, 0.5280, 0.3250, 0.4610,
         0.3431, 0.4942]], device='cuda:1', grad_fn=<SigmoidBackward0>)
tensor([-0.7198,  0.0596, -0.1184, -0.8253,  0.0068, -0.5731,  1.1710, -0.2092,
         0.8761, -0.9664, -0.6095,  0.2866,  0.0716,  0.0924,  3.1012,  0.1120,
        -0.7310, -0.1563, -0.6494, -0.0233], device='cuda:1',
       grad_fn=<SelectBackward0>)
tensor(-0.7198, device='cuda:1', grad_fn=<UnbindBackward0>)
tensor(0.0596, device='cuda:1', grad_fn=<UnbindBackward0>)
tensor(-0.1184, device='cuda:1', grad_fn=<UnbindBackward0>)
tensor(-0.8253, device='cuda:1', grad_fn=<UnbindBackward0>)
tensor(0.0068, device='cuda:1', grad_fn=<UnbindBackward0>)
tensor(-0.5731, device='cuda:1', grad_fn=<UnbindBackward0>)
tensor(1.1710, device='cuda:1', grad_fn=<UnbindBackward0>)
tensor(-0.2092, device='cuda:1', grad_fn=<UnbindBackward0>)
tensor(0.8761, device='cuda:1', g

In [ ]:
image, label = train_set[23]
print(image, label)

In [ ]:
for step,(data, labels) in enumerate(multi_voc_ValLoader):
    data = data.to(device) 
    labels = labels.to(device)